In [30]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.preprocessing import OneHotEncoder

In [47]:
iris = load_iris()
X = iris.data
y = iris.target 
X_df = pd.DataFrame(X, columns=iris.feature_names)
y_df = pd.DataFrame(y, columns=["species"])

In [48]:
encoder = OneHotEncoder(sparse_output=False)
y_one_hot = encoder.fit_transform(y_df)
y_one_hot_df = pd.DataFrame(y_one_hot, columns=encoder.get_feature_names_out(["species"]))
print(y_one_hot_df)


     species_0  species_1  species_2
0          1.0        0.0        0.0
1          1.0        0.0        0.0
2          1.0        0.0        0.0
3          1.0        0.0        0.0
4          1.0        0.0        0.0
..         ...        ...        ...
145        0.0        0.0        1.0
146        0.0        0.0        1.0
147        0.0        0.0        1.0
148        0.0        0.0        1.0
149        0.0        0.0        1.0

[150 rows x 3 columns]


In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42,shuffle=True)

In [50]:
@tf.function
def predict(X, W, b):
    return tf.nn.softmax(tf.matmul(X, W) + b)

In [51]:
@tf.function
def L(y, y_hat):
    loss = y*tf.math.log(y_hat)
    return -tf.reduce_mean(tf.reduce_sum(loss, axis=1)) #ma tran cong tung dong lai 

In [52]:
X_train = tf.constant(X_train)
y_train = tf.constant(y_train)
X_test = tf.constant(X_test)
y_test = tf.constant(y_test)


In [56]:
alpha = 0.01
W = tf.Variable(tf.random.normal(shape=(4, 3), mean=0.0, stddev=1.0, dtype=tf.float64)) 
b = tf.Variable(tf.random.normal(shape=(3,), mean=0.0, stddev=0.01, dtype=tf.float64)) 
    
for it in range(350):
    with tf.GradientTape() as t:
        current_loss = L(y_train, predict(X_train, W, b))
    print("it", it, current_loss)
    dW, db = t.gradient(current_loss, [W, b])
    W.assign_sub(alpha * dW)
    b.assign_sub(alpha * db)
        


it 0 tf.Tensor(5.501504851751675, shape=(), dtype=float64)
it 1 tf.Tensor(5.1706682343743955, shape=(), dtype=float64)
it 2 tf.Tensor(4.845811502080669, shape=(), dtype=float64)
it 3 tf.Tensor(4.529388882537142, shape=(), dtype=float64)
it 4 tf.Tensor(4.224386049876364, shape=(), dtype=float64)
it 5 tf.Tensor(3.934041962449851, shape=(), dtype=float64)
it 6 tf.Tensor(3.6613269029416937, shape=(), dtype=float64)
it 7 tf.Tensor(3.4083133206755645, shape=(), dtype=float64)
it 8 tf.Tensor(3.1757197695961765, shape=(), dtype=float64)
it 9 tf.Tensor(2.9628573567680356, shape=(), dtype=float64)
it 10 tf.Tensor(2.767966365616568, shape=(), dtype=float64)
it 11 tf.Tensor(2.588726583193956, shape=(), dtype=float64)
it 12 tf.Tensor(2.422717317964619, shape=(), dtype=float64)
it 13 tf.Tensor(2.267722362828175, shape=(), dtype=float64)
it 14 tf.Tensor(2.121884032894659, shape=(), dtype=float64)
it 15 tf.Tensor(1.9837555294557812, shape=(), dtype=float64)
it 16 tf.Tensor(1.8523003426171605, shape=()

In [57]:
y_pred_test = predict(X_test, W, b)
    
y_pred_labels = tf.argmax(y_pred_test, axis=1).numpy()
y_true_labels = tf.argmax(y_test, axis=1).numpy()
print(y_pred_labels)
print(y_true_labels)
    
accuracy = np.mean(y_pred_labels == y_true_labels)
    
print(f'Accuracy: {100*accuracy}%')

[1 0 2 1 1 0 1 2 2 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 1 2 2 2 0 0]
[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]
Accuracy: 93.33333333333333%
